In [6]:
# оконтуривание

import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('/Users/admin/Desktop/pictures/coins2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Noise removal
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,kernel, iterations = 2)

# Sure background area
sure_bg = cv2.dilate(opening, kernel, iterations = 3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Now, mark the region of unknown with zero
markers[unknown == 255] = 0
markers = cv2.watershed(img, markers)
img[markers == -1] = [255,0,0]
cv2.imwrite('/Users/admin/Desktop/pictures/output_watershed.jpg', img)

#cv2.imshow('after watershed:', img)

True

In [16]:
# сегментация

# First we load image, convert it to grayscale and threshold it with a suitable value 
# Taken Otsu's binarization, so it would find the best threshold value
import cv2
import numpy as np

img = cv2.imread('/Users/admin/Desktop/pictures/kitten.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Now we have to create the marker
# Marker is the image with same size as the original one

# Foreground region:
# Erode main objects a little bit to be sure remaining region belongs to foreground
fg = cv2.erode(thresh, None, iterations = 2)

# Background region:
# Dilate the thresholded image so that background region is reduced
# But we are sure remaining black region is 100% background. Set it to 128
bgt = cv2.dilate(thresh, None, iterations = 3)
ret,bg = cv2.threshold(bgt, 1, 128, 1)

# Now we add both fg and bg
marker = cv2.add(fg, bg)

# Convert it into 32SC1
marker32 = np.int32(marker)

# Finally apply watershed
cv2.watershed(img, marker32)

# Convert result back into uint8 image
m = cv2.convertScaleAbs(marker32)

#ret,thresh = cv2.threshold(m, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#res = cv2.bitwise_and(img, img, mask = thresh)

# Save the result
cv2.imwrite('/Users/admin/Desktop/pictures/output_watershed.jpg', m)

# https://stackoverflow.com/questions/11294859/how-to-define-the-markers-for-watershed-in-opencv

True